In [ ]:
import os
import pandas as pd
import numpy as np
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

%load_ext autoreload
%autoreload explicit

%aimport opendata_connect

# Retrieve data
___
Sourced from Golden Cheetah Open Data Project

In [ ]:
try:
    if 'apple' in os.environ['BUILD']:
        root_dir = '../gc_opendata-samples'
    else:
        root_dir = 'E:\gc_opendata'
except:
    root_dir = 'E:\gc_opendata'

In [ ]:
od = opendata_connect.open_dataset(root_dir)
ov = od.get_athlete_summary(od.athlete_ids[2])
ov.columns = [col.replace('METRICS.','') for col in ov.columns]
l = ov[ov['sport'] == 'Bike']

In [ ]:
l[['60m_critical_power'
,'20m_critical_power']].rolling(window=7,min_periods=1).max().plot(figsize=(10,2))

# Sample Explore

In [ ]:
target_id = "7567ac31-d479-4b25-9538-541ae8085288"
act_samp = activ_sum[activ_sum['id'] == target_id].copy()
act_samp.columns.tolist()

In [ ]:
act_samp['athlete_weight'] = act_samp['20m_critical_power']/act_samp['20m_peak_wpk']
act_samp['athlete_weight'] = act_samp['athlete_weight'].ffill().bfill()
act_samp.set_index('date',inplace=True)
act_samp[act_samp['sport'] == 'Bike']['20m_critical_power'].rolling(window=90, min_periods=1).max().plot();
act_samp.plot(kind='scatter',x='workout_time',y='average_hr')

In [ ]:
def calc_vo2(row, max_hr, resting_hr):
    if row['sport'] == 'Bike':
        percent_vo2 = (row['average_hr'] - resting_hr)/(max_hr - resting_hr)
        vo2_estimated = (((row['average_power']/75)*1000)/row['athlete_weight']) / percent_vo2
        # vo2_estimated = row['VO2max_Detected'] ## overide with Garmin VO2 Estimation
    elif row['sport'] == 'Run':
        percent_vo2 = (row['average_hr'] - resting_hr)/(max_hr - resting_hr)
        vo2_estimated = (210/row['average_speed']) / percent_vo2
    else:
        vo2_estimated =  0
    return vo2_estimated
act_samp['VO2'] = act_samp.apply(lambda row: calc_vo2(row, 179, 45), axis=1)

# Modeling

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class TimeSeriesCNN(nn.Module):
    def __init__(self):
        super(TimeSeriesCNN, self).__init__()
        
        # Define the layers of the CNN
        self.conv1 = nn.Conv1d(in_channels=8, out_channels=32, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.fc1 = nn.Linear(in_features=64*8, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=2)

    def forward(self, x):
        # Pass the input through the layers of the CNN
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64*8)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
l

In [ ]:
input_data = fl_df[['TIZ1_3','TIZ2_3','TIZ3_3','VO2_l1']].to_numpy()[:-1]
target_data = fl_df['VO2'][:-1].to_numpy()

In [ ]:
test_percent = 0.8
test_split = int(input_data.shape[0] * test_percent)
train_input_data, train_target_data = input_data[:test_split], target_data[:test_split]
test_input_data, test_target_data = input_data[test_split:], target_data[test_split:]

In [ ]:
model = TimeSeriesCNN()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    inputs = torch.from_numpy(train_input_data[:32].T).float()
    targets = torch.from_numpy(train_target_data).float()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    # Evaluate the model on the test set
    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            inputs = torch.from_numpy(test_data).float()
            targets = torch.from_numpy(test_labels).float()
            outputs = model(inputs)
            test_loss = criterion(outputs, targets)
            print(f"Epoch {epoch}: Test loss = {test_loss.item()}")

In [ ]:
# Generic NN
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2)
        self.fc3 = nn.Linear(2, 1)
         
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=1)
    
net = NN()
print(net)